# Particle Trapping with Muon decay

Notes:
- Simulation is a two stage deal:
    - Generate muons isotropically within the DS trap region, allow them to propigate for 700 ns (no decay)
    - Use that output to further evolve remaining muons, allowing them to decay to electrons
- Their are two copies of each 2nd stage muon in `ntpart`.
    - Looks like copy 1 has t==0, p==100, pstop=p at 700ns
    - The second muon has t=~700, p==100, parent_pstop = original p_stop
- No muons have an initial timestamp > 700 ns
- No electrons are born before 700 ns


### Some Findings

Number of trapped muons per init muon p:
* p=80 : 22317*100 = ~2.2mil
* p=100: 30241*100 = ~3mil
* p=110: 34999*100 = ~3.5mil
* p=140: 19031*100 = ~1.9mil

Number of unique events in which an electron with p>100 makes it farther upstream than 11200mm:
* Muon init p=80: 0 Events
* Muon init p=100: 2 Events
* Muon init p=110: 5 Events
* Muon init p=140: 21 Events

In [ ]:
from mu2e import mu2e_ext_path
from mu2e.dataframeprod import g4root_to_df
from root_pandas import read_root
import pandas as pd
from mu2e.mu2eplots import mu2e_plot3d_ptrap
from mu2e.mu2eplots import mu2e_plot3d_ptrap_traj
from mu2e.mu2eplots import mu2e_plot3d_ptrap_anim
import cPickle as pkl
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
input_root_skim = mu2e_ext_path+'datafiles/G4ParticleSim/stage2_140p/iso_muons_GA05_stage2_vd_combo_skim.root'
df_ntpart = read_root(input_root_skim, 'ntpart')
df_nttvd = read_root(input_root_skim, 'nttvd')
df_ntvd = read_root(input_root_skim, 'ntvd')

In [ ]:
input_root_single = mu2e_ext_path+'datafiles/G4ParticleSim/stage2_140p/iso_muons_GA05_stage2_vd0.root'
df_ntpart = read_root(input_root_single, 'readvd/ntpart')

In [ ]:
df_ntpart.query('pdg==11 and parent_pdg==13').pz.hist(bins=30)
plt.title('Initial Electron Pz (Muon parent p = ~140 MeV)')
plt.ylabel('Counts')
plt.xlabel('Pz (MeV)')

In [ ]:
df_ntpart.query('pdg==13 and time>0 and pstop>100').pzstop.hist(bins=50, color='r')
plt.title('Muon Pz (t = 700 ns)')
plt.ylabel('Counts')
plt.xlabel('Pz (MeV)')

In [ ]:
store = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/iso_muons_GA05_stage2_vd_solo.h5')
df_nttvd_solo = store.df_nttvd
df_ntvd_solo = store.df_ntvd
df_ntpart_solo = store.df_ntpart
store.close()

In [ ]:
store_xray = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/low_e_ele_0T_v580.h5')
df_xray = store_xray.df_ntpart
store_xray.close()

In [ ]:
df_ntpart.query('pdg==11 and time>0')[0:20000]

In [ ]:
df_mu = df_ntpart.query('pdg==13 and time>0')[0:20000].reset_index()
#df_mu = df_ntpart.query('pdg==13 and time>0').reset_index()

df_mu.name = 'Muons'
mu2e_plot3d_ptrap(df_mu,
                  'z','x','y', x_range=[3700,17500], y_range = [-1000,1000], z_range=[-1000,1000], save_name=None,
                   df_xray=df_xray, color='p', title='Muon position at t=700ns')

In [ ]:
bads_runevt = df_nttvd.query('pdg==11 and z<11200 and p>85').runevt.unique()
#df_mu = df_ntpart.query('pdg==13 and time>0 and 100<xstop<575 and 150>ystop>-600')[0:20000].reset_index()
df_mu_bads = df_ntpart[df_ntpart.runevt.isin(bads_runevt)]
#df_mu = df_ntpart.query('pdg==13 and time>0').reset_index()

df_mu_bads.name = 'Muons'
mu2e_plot3d_ptrap(df_mu_bads,
                  'zstop','xstop','ystop', x_range=[3700,17500], y_range = [-1000,1000], z_range=[-1000,1000], save_name=None,
                   df_xray=df_xray, color='pstop', title='Muon Decay Location Yeilding Sneaky Electrons')

In [ ]:
df_mu = df_ntpart.query('pdg==13 and time>0')
df_el = df_ntpart.query('pdg==11')

df_mu.pz.hist(range=[-5,5],bins=100)

In [ ]:
df_el.p.hist(bins=50)

In [ ]:
df_ntvd_ele = df_ntvd.query('pdg==11').reset_index()
df_ntvd_ele.name = 'Electrons'
mu2e_plot3d_ptrap(df_ntvd_ele,
                  'z','x','y', x_range=[3700,17500], y_range = [-1000,1000], z_range=[-1000,1000], save_name=None,
                   df_xray=df_xray, color='p', symbol='x', title='Electrons, p>100 VD hits')

In [ ]:
len(df_ntvd_ele.query('pdg==11 and z<11200').runevt.unique())

In [ ]:
init_mu = len(df_ntpart.query('pdg==13 and time==0'))
ele_100_110 = len(df_ntpart.query('pdg==11 and 100<p<110'))
ele_103_105 = len(df_ntpart.query('pdg==11 and 103.85<p<105.1'))
print init_mu
print ele_100_110, (float(ele_100_110)/init_mu)
print ele_103_105, (float(ele_103_105)/init_mu)

In [ ]:
df_nttvd_ele_solo = df_nttvd_solo.query('runevt==133139712 and pdg==11')
df_nttvd_mu_solo = df_nttvd_solo.query('runevt==133139712 and pdg==13')
df_nttvd_ele_solo.name = 'Sneaky Electron'

In [ ]:
df_ntvd_ele_solo = df_ntvd.query('runevt==621228021543 and pdg==11')

In [ ]:
df_ntvd.query('runevt==621228021543 and pdg==11')

In [ ]:
g.plot(fig)

In [ ]:

g, fig = mu2e_plot3d_ptrap_anim(df_nttvd_ele_solo,'z','x','y',df_xray,
                                 color=True,title='Electrons from trapped muons')

In [ ]:
df_ntvd_ele_solo.name = 'Sneaky Electron'
df_ntvd_ele_solo.sort_values('time', inplace=True)

In [ ]:
df_tmp = df_nttvd.query('runevt==2').sort_values('time')
df_tmp.name='Cosmic'

In [ ]:
df_nttvd_solo.name = 'Electron'

In [ ]:
mu2e_plot3d_ptrap_traj(df_nttvd_ele_solo,'z','x','y',df_xray=df_xray,
                        title='Electrons from trapped muons')

In [ ]:
df_nttvd_mu_test = df_nttvd.query('runevt==38654 and pdg==13')
df_nttvd_mu_test.name = 'Muon'

In [ ]:
df_ntvd_ele.query('p>=100 and z<11090')

In [ ]:
df_ntvd_ele[df_ntvd_ele.z==df_ntvd_ele.z.min()]

In [ ]:
df_nttvd[df_nttvd.runevt==58133139712]

In [ ]:
df_ntvd.query('pdg==11 and z<11800').p.hist()
plt.title('Electrons, Z<11800')
plt.xlabel('p (MeV)')

In [ ]:
df_ntvd.query('p>=100').z.hist()
plt.title('Electrons, p>100')
plt.xlabel('Z (mm)')

In [ ]:
df_nttvd_normal[df_nttvd_normal.sid==1].pz.hist(bins=40)

In [ ]:
df_ntpart.query('pdg==11').p.hist(bins=20)
plt.title('Electron Initial Momentum')
plt.xlabel('p (MeV)')

In [ ]:
df_ntpart.query('pdg==11 and parent_p>75').z.hist(bins=20, )
plt.title('Electron Starting Z (muon p>75)')
plt.xlabel('Z (mm)')

In [ ]:
df_ntvd.query('pdg==11 and parent_p>75').z.hist(bins=20, )
plt.title('Electron Stoping Z (muon p>75)')
plt.xlabel('Z (mm)')

In [ ]:
df_ntpart.columns

In [ ]:
df_ntpart.p.hist(bins=50)
plt.xlabel('Momentum (MeV)')
plt.ylabel('Count')
plt.title('Initial Muon Momentum for `ntpart` tree')

In [ ]:
g.save?

In [ ]:
df_nttvd.query('pdg==13').sid.max()

In [ ]:
from mu2e.dataframeprod import g4root_to_df
g4root_to_df(mu2e_ext_path+'datafiles/G4ParticleSim/iso_muons_GA05_stage2_vd2',True,True)

In [ ]:
df_ntpart[df_ntpart.evt==25456]

## Some results

In [ ]:
print 'total number of muons (that survive 700)', len(df_ntpart.query('pdg==13 and time==0'))
print 'total number of electrons (between 700 and 10k)', len(df_ntpart.query('pdg==11'))

In [ ]:
df_ntpart.query('pdg==13 and time==0').pstop.hist()
plt.gca().set_yscale('log')
plt.title('Muon momenta at 700 ns')


In [ ]:
df_ntpart.query('pdg==13 and time==0').zstop.hist()
plt.gca().set_yscale('log')
plt.title('Muon z position at 700 ns')

In [ ]:
df_ntpart.query('pdg==11').p.hist()
plt.title('Electron starting momenta')

In [ ]:
from mu2e.dataframeprod import g4root_to_df

g4root_to_df(mu2e_ext_path+'datafiles/G4ParticleSim/tmp/iso_muons_GA05_stage2_vd1',True,True)
g4root_to_df(mu2e_ext_path+'datafiles/G4ParticleSim/tmp/iso_muons_GA05_stage2_vd2',True,True)

In [ ]:
store = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/tmp/iso_muons_GA05_stage2_vd1.h5')
df_ntpart1 = store.df_ntpart
store.close()
store = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/tmp/iso_muons_GA05_stage2_vd2.h5')
df_ntpart2 = store.df_ntpart
store.close()

In [ ]:
df_ntpart1.query('runevt==28139').pstop

In [ ]:
df_ntpart2.query('runevt==28139').pstop

# Bookkeeping for high-stats samples

In [ ]:
from mu2e.dataframeprod import g4root_to_df
import tqdm
for i in range(100):
    g4root_to_df(mu2e_ext_path+'datafiles/G4ParticleSim/stage2_80p/iso_muons_GA05_stage2_vd{}'.format(i),True,True,cluster=i)

In [ ]:
store = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/stage2/iso_muons_GA05_stage2_vd0.h5')
df_nttvd = store.df_nttvd
df_ntvd = store.df_ntvd
df_ntpart = store.df_ntpart
store.close()


In [ ]:
from root_pandas import to_root, read_root
import tqdm
df_nttvd_list = []
for i in tqdm.tqdm_notebook(range(100)):
    input_root = mu2e_ext_path+'datafiles/G4ParticleSim/stage2_110p/iso_muons_GA05_stage2_vd{}.root'.format(i)
    output_root = mu2e_ext_path+'datafiles/G4ParticleSim/stage2_80p/iso_muons_GA05_stage2_vd{}_skim.root'.format(i)
    df_ntpart = read_root(input_root, 'ntpart')
    df_nttvd = read_root(input_root, 'nttvd')
    df_ntvd = read_root(input_root, 'ntvd')
    
    good_runevt = df_ntpart.query('pdg==11 and p>75').runevt.unique()
    df_ntpart = df_ntpart[df_ntpart.runevt.isin(good_runevt)]
    df_ntvd = df_ntvd[df_ntvd.runevt.isin(good_runevt)]
    df_nttvd = df_nttvd[df_nttvd.runevt.isin(good_runevt)]
    df_ntvd.to_root(output_root, tree_key='ntvd', mode = 'w')
    df_nttvd.to_root(output_root, tree_key='nttvd', mode = 'a')
    df_ntpart.to_root(output_root, tree_key='ntpart', mode = 'a')


In [ ]:
df_nttvd = pd.concat(df_nttvd_list, ignore_index=True)


In [ ]:
store = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/stage2/iso_muons_GA05_stage2_vd_combo.h5')

In [ ]:
import root_pandas

In [ ]:
from mu2e.dataframeprod import g4root_to_df
from mu2e import mu2e_ext_path
df_ntpart, df_nttvd, df_ntvd = g4root_to_df(mu2e_ext_path+'datafiles/G4ParticleSim/stage2/iso_muons_GA05_stage2_vd_combo',tree_prefix='')

In [ ]:
input_root_solo = mu2e_ext_path+'datafiles/G4ParticleSim/iso_muons_GA05_stage2_vd_solo'
g4root_to_df(input_root_solo, do_basic_modifications=True, make_pickle=True)

In [ ]:
#store = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/z13k_muons_extmat_GA05.h5')
store = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/iso_muons_GA05_stage2_vd2.h5')
df_nttvd = store.df_nttvd
df_ntvd = store.df_ntvd
df_ntpart = store.df_ntpart
store.close()
store_xray = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/low_e_ele_0T_v580.h5')
df_xray = store_xray.df_ntpart
store_xray.close()
#store2 = pd.HDFStore(mu2e_ext_path+'datafiles/G4ParticleSim/z13k_muons_nomat_GA05.h5')
#df_nttvd2 = store2.df_nttvd
#df_ntpart2 = store2.df_ntpart
#store2.close()

In [ ]:
from root_pandas import to_root

In [ ]:
from mu2e.dataframeprod import g4root_to_df_skim_and_combo
g4root_to_df_skim_and_combo(mu2e_ext_path+'datafiles/G4ParticleSim/stage2_110p_5k/iso_muons_GA05_stage2_vd', 100)

In [ ]:
_, df_nttvd, _ = g4root_to_df(mu2e_ext_path+'datafiles/G4ParticleSim/traps_vd', trees=['tvd'], do_basic_modifications=True)

In [ ]:
df_tmp = df_nttvd.query('runevt==1 and pdg==13').sort_values('time')
df_tmp.name='Cosmic'

In [ ]:
df_tmp

In [ ]:
mu2e_plot3d_ptrap_traj(df_tmp,'z','x','y',df_xray=df_xray, z_range=(-1000, 8000),
                        title='Cosmic muons')

In [ ]:
df_nttvd.time.max()